In [1]:
cd ../

/nfs/home/dgranziol/kfac-curvature


In [2]:
import argparse
import time
import tabulate

import numpy as np
import matplotlib.pyplot as plt

from gpytorch.utils.lanczos import lanczos_tridiag

import torch

from curvature import data, models, losses, utils
from curvature.methods.swag import SWAG

Older torchvision found


/nfs/home/dgranziol/anaconda2/envs/newtensor/lib/python3.7/site-packages/torchvision/transforms/transforms.py:396: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [14]:
dataset = 'MNIST'
data_path = '/nfs/home/dgranziol/kfac-curvature/data'
#LOGISTIC REGRESSION MNIST
ckpt = '/nfs/home/dgranziol/kfac-curvature/out/MNIST/Logistic/SGD/seed=1_lr=0.01_mom=0.9_wd=0.0_numepochs=1000/checkpoint-01000.pt'

dataset = ckpt.split('/out/')[1].split('/')[0]
model = ckpt.split('/seed=')[0].split('/')[7]
network = model
epochsave = int(ckpt.split('checkpoint-')[1].split('.')[0])

# #MLP small MNIST
# ckpt = '/nfs/home/dgranziol/kfac-curvature/out/MNIST/MLP_sdp/SGD/seed=1_lr=0.03_mom=0.9_wd=0.0_batchsize=128_numepochs=200/checkpoint-00200.pt'


#model = 'Logistic'
print('Using model %s' % model)
#from curvature.models import logistic_regression
model_cfg = getattr(models,model)

Using model Logistic


In [15]:
batch_size = 5000

datasets, num_classes = data.datasets(
    dataset,
    data_path,
    transform_train=model_cfg.transform_test,
    transform_test=model_cfg.transform_test,
    use_validation=False,
    train_subset=None,
    train_subset_seed=None,
)
loader = torch.utils.data.DataLoader(
    datasets['train'],
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
full_datasets, _ = data.datasets(
    dataset,
    data_path,
    transform_train=model_cfg.transform_train,
    transform_test=model_cfg.transform_test,
    use_validation=False,
)
full_loader = torch.utils.data.DataLoader(
    full_datasets['train'],
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

Loading MNIST from /nfs/home/dgranziol/kfac-curvature/data
You are going to run models on the test set. Are you sure?
Using train (60000) + test (10000)
Loading MNIST from /nfs/home/dgranziol/kfac-curvature/data
You are going to run models on the test set. Are you sure?
Using train (60000) + test (10000)


In [16]:
model = model_cfg.base(*model_cfg.args, num_classes=num_classes, **model_cfg.kwargs)
print('Loading %s' % ckpt)
checkpoint = torch.load(ckpt)
model.load_state_dict(checkpoint['state_dict'])

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device = torch.device('cpu')
model.to(device)
num_parametrs = sum([p.numel() for p in model.parameters()])
criterion = losses.cross_entropy


Loading /nfs/home/dgranziol/kfac-curvature/out/MNIST/Logistic/SGD/seed=1_lr=0.01_mom=0.9_wd=0.0_numepochs=1000/checkpoint-01000.pt


In [17]:
#getting the full Hessian
model.zero_grad()
N = len(loader.dataset)
a = torch.zeros(num_parametrs,num_parametrs).cpu()
for input, target in full_loader:
    #input = input.cuda(non_blocking=True)
    #target = target.cuda(non_blocking=True)
    loss, _, _ = criterion(model, input, target)
    loss *= input.size()[0] / N
    print(loss)
    #loss.backward(retain_graph=True)
    #loss.backward()
    grad_list = torch.autograd.grad(loss, model.parameters(), create_graph=True)
    grad_i = torch.cat([g.view(-1) for g in grad_list]).cpu()
    for i in range(0,num_parametrs):
        #grad_list2 = torch.autograd.grad(grad_i[i], model.parameters(), create_graph=True)
        a[i] = torch.cat([g.view(-1) for g in torch.autograd.grad(grad_i[i], model.parameters(), create_graph=True)]).cpu()

tensor(0.0162, grad_fn=<MulBackward0>)
tensor(0.0174, grad_fn=<MulBackward0>)
tensor(0.0192, grad_fn=<MulBackward0>)
tensor(0.0159, grad_fn=<MulBackward0>)
tensor(0.0181, grad_fn=<MulBackward0>)
tensor(0.0174, grad_fn=<MulBackward0>)
tensor(0.0183, grad_fn=<MulBackward0>)
tensor(0.0192, grad_fn=<MulBackward0>)
tensor(0.0180, grad_fn=<MulBackward0>)
tensor(0.0186, grad_fn=<MulBackward0>)
tensor(0.0180, grad_fn=<MulBackward0>)
tensor(0.0201, grad_fn=<MulBackward0>)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f171b63cc80>
Traceback (most recent call last):
  File "/nfs/home/dgranziol/anaconda2/envs/newtensor/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/nfs/home/dgranziol/anaconda2/envs/newtensor/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 893, in _shutdown_workers
    self.worker_result_queue.put((None, None))
  File "/nfs/home/dgranziol/anaconda2/envs/newtensor/lib/python3.7/multiprocessing/queues.py", line 87, in put
    self._start_thread()
  File "/nfs/home/dgranziol/anaconda2/envs/newtensor/lib/python3.7/multiprocessing/queues.py", line 170, in _start_thread
    self._thread.start()
  File "/nfs/home/dgranziol/anaconda2/envs/newtensor/lib/python3.7/threading.py", line 847, in start
    _start_new_thread(self._bootstrap, ())
RuntimeError: can't start new thread


RuntimeError: DataLoader worker (pid 24702) is killed by signal: Killed. 

In [ ]:
num_parametrs

In [ ]:
matrix = a.detach().numpy()
C = np.linalg.svd(matrix, full_matrices=True)
print(C[1][0:30])

In [ ]:
import scipy.io as sio
sio.savemat('matrix_'+dataset+'_'+network+'_'+str(epochsave)+'.mat', {'matrix':matrix})

print('matrix_'+dataset+'_'+network+'_'+str(epochsave)+'.mat saved')

import matplotlib
#ax = sns.heatmap(matrix, yticklabels=2, xticklabels=False)
#uniform_data = np.random.rand(10000, 10000)
plt.imshow(matrix)
cax = plt.axes([0.85, 0.1, 0.075, 0.8])
plt.colorbar(cax=cax)
plt.savefig('matrix_'+dataset+'_'+network+'_'+str(epochsave)+'.pdf',bbox_inches='tight')
plt.show()

In [ ]:
#Eigenvalue distribution
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=C[1], bins=10000, color='#0504aa',
                            alpha=0.7, rwidth=0.001)
# plt.grid(axis='y', alpha=0.75)
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.title('My Very Own Histogram')
# plt.text(23, 45, r'$\mu=15, b=3$')
# maxfreq = n.max()
# # Set a clean upper y-axis limit.
# plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)


In [ ]:
counter = 0
threshold = 1e-4
for i in range(0,len(C[1])):
    if C[1][i] < threshold:
        counter = counter+1
print(counter)

In [ ]:
len(matrix[0])

In [ ]:
[max(a) for a in matrix]

In [ ]:
plt.figure(ax)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model.to(device)

In [ ]:
class CurvVecProduct(object):
    def __init__(self, loader, model, criterion, curvature_matrix, full_loader=None):
        self.loader = loader
        self.full_loader = full_loader
        self.model = model
        self.criterion = criterion
        self.iters = 0
        self.timestamp = time.time()
        self.curvature_matrix = curvature_matrix

    def __call__(self, vector):
        start_time = time.time()
        if self.curvature_matrix == 'hessian':
            output = utils.hess_vec(
                vector,
                self.loader,
                self.model,
                self.criterion,
                cuda='cuda'
               
                )
        elif self.curvature_matrix == 'covgrad':
            output = utils.covgrad_vec(
                vector,
                self.loader,
                self.model,
                self.criterion,
                cuda='cuda',
            )
        else:
            raise ValueError("Unrecognised curvature_matrix argument "+self.curvature_matrix)
        time_diff = time.time() - start_time

        self.iters += 1
        print('Iter %d. Time: %.2f' % (self.iters, time_diff))
        # return output.unsqueeze(1)¬
        return output.cpu().unsqueeze(1)

w = torch.cat([param.detach().cpu().view(-1) for param in model.parameters()])
w_l2_norm = torch.norm(w).numpy()
w_linf_norm = torch.norm(w, float('inf')).numpy()

In [ ]:
#Hessian
productor = CurvVecProduct(full_loader, model, criterion, 'hessian')
#utils.bn_update(loader, model)
Q, T = lanczos_tridiag(productor, 15, dtype=torch.float32, device='cpu', matrix_shape=(num_parametrs, num_parametrs))

eigvals, eigvects = T.eig(eigenvectors=True)
gammas = eigvects[0, :] ** 2
V = eigvects.t() @ Q.t()

In [ ]:
eigval = []
for i in range(0,len(eigvals)):
    eigval.append(eigvals[i][0])
mean = np.dot(eigval,gammas)
print(mean)
eigval, gammas

In [ ]:
grad = utils.grad(loader, model, criterion, cuda='cuda' == 'cuda', bn_train_mode=False)

#Grad distribution
import matplotlib.pyplot as plt

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=grad.cpu().numpy(), bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=torch.cat((checkpoint['state_dict']['layer.weight'].view(-1),checkpoint['state_dict']['layer.bias']),0).cpu().numpy(), bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

